# 텍스트 마이닝 프로젝트 (2020.5~)
### Data Crawling
#### 휴먼지능정보공학과 류지혜, 이지민

## 에어팟 1세대

In [19]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

#에어팟 1세대 검색 (2세대 검색하고싶으면 주소바꾸셈)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=10776906666&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25201%25EC%2584%25B8%25EB%258C%2580&NaPm=ct%3Dkb3ef8sg%7Cci%3Df7d5699aba286396ee3757965561aa228125c722%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Da9cfdf21495248f435f4ae3cbea263c694784ae1"
#xpath
shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[21]/a/span" #배터리 소모

# name="에어팟 1세대" #에어팟 2세대 검색할 때 2세대로 바꾸셈
# category1_="음질"
# category2_="품질"
# category3_="성능"
# category4_="기능"
# category5_="배터리 수명"
# category6_="배터리 소모" 

header = {'User-Agent': ''}
d = webdriver.Chrome(executable_path=r"C:\Users\jmlee\Code\chromedriver_win32\chromedriver.exe") # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

In [20]:
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

In [47]:
# .to_csv 
# 최초 생성 이후 mode는 append
import pandas as pd
import os
def save():
    if not os.path.exists('output.csv'):
        df1.to_csv('output.csv', mode='w',encoding='utf-8')
    else:
        df1.to_csv('output.csv',mode='a',encoding='utf-8',header=False)

In [21]:
#"음질" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name=name[0]  #에어팟 1세대
category=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name,category,reviews,stars,cnt)
save()

페이지 1 

1 배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 이상 없이 잘 됐어요!~연결도 잘되고, 음악 잠깐 들었는데 음질도 좋고 정말 편해요^^미리 구입해둔 케이스도 장착해서 바로 사용했어요~가격도 L.POINT랑 청구할인 받아서 직구보다 저렴하게 잘 구입했습니다.좋은 상품 감사합니다:) 5 

2 아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아이패드로 바꿔서 들을때 불편해서 블루투스 이어폰을 검색하다 주변의 구매자들 모두 하나같이 최고다라고 이야기 한 애플의 에어팟을 구매하였습니다.

할인쿠폰과 카드청구할인 혜택으로 저렴하게 구입할 수 있었고 배송도 공휴일이 포함되었음에도 2일만에 받을 수 있어 좋았습니다.

제품은 박스를 에어캡으로 감싸고 다시 큰 박스에 넣어 안전하게 배송이 되었습니다. 제품도 2018년 5월 제조 최신 제품으로 받았습니다.

받자마자 아이폰에 연결하고 사용해봤는데 . 대단하 만족합니다. 5 

3 다른 저렴한 곳도 있었습니다.

그래도 제조일자 최신에 페이백으로 다시 받는 금액 생각해보니 얼추 괜찮다고 생각해서 구매했습니다.

사용해보니 정말 편합니다. 음질도 나쁘지 않구요.
귀에 꼽고 장시간 있으면 혹시 떨어졌나 하는 생각도 들구요 가끔씩 귀 한번 만집니다 ㅋㅋㅋ
적응하면 신경 안쓸듯합니다.너무 불안하면 실리콘 줄 하나 구매해서 쓰려고 합니다 4 

4 택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사했어요
친구들 말만듣고 좋구나...생각만 했는데
막상 사용하니 너무 좋아요 어제 노래들으면서 청소 했는데 삶의 질이 달라졌다는 말이 생각 나더라구요 ㅎㅎ 너무 마음에 들어요 그리고 차에서 선이 없으니까 걸리적 거리는게 없어서 너무 편했습니다 음질도 짱짱하구요 충전도 빠르게 되는거 같고 너무 좋았습니다. 5 

5 배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 받았어요. 귀에서 빠지면 어떡하나 

32 하이마트가 혜택이 많아서 구매했는데 배송도 빠르네요. 월 오후 주문이 화 오후에 도착했어요! 집에서 기다렸는데 택배기사님이 연락이나 초인종 없이 문앞에 놓고가셔서 하염없이 기다리긴 했습니다ㅜㅜ
우선 제품은 정말 만족하고 매일 잘 쓰고있어요. 근데 지지직 소리나 음악 채널 변경 시 틱틱 하는 잡음이 좀 있어서 며칠 더 써보다가 서비스센터에 문의해보려해요. 5 

33 음질은 아무래도 유선보다는 약간은 떨어집니다. 하지만 애플 이어팟(유선)과 비교하면 저음부가 풍부해서 어느정도 음질을 보완했다는 생각이 드네요.연결은 매우 빠르고 편리합니다. 애플기기가 2대 이상일때 전환도 매우매우 빠른 점도 좋습니다. 여기에 에어팟을 귀에 착용하면 자동으로 인식하고, 귀에서 빼면 음악이 알아서 정지되는 것도 편하네요.또, 선이 없어져서 운동할때 엄청 편하고 귀에서 잘 안빠집니다. 단순한 음향기기를 구매했다는 관점에서 보면 실망스러울 수도 있는데(비슷한 가격대에 음질 더 좋은 유선 이어폰들이 있으니), 그런걸 떠나서 생활이 꽤 편리해진다는 점이 아주 좋아요. 5 

34 계속 고민하다가 사야겠다 싶어서... 엘페이 등록하고 신한카드 청구할인으로 퀵배송으로 5천원 쓰고 받았습니다. 강남이어서 그런것인지 주문하고 30분 만에 왔어요! 완전 기분 짱이네요 ㅎㅎ 음질 좋고 간편한 연결에 짱입니다. 왜 이렇게 많이들 쓰는지 알겠네요. 5 

35 처음에 출시됐을때는 뭐야? 하는 반응으로 거들떠 보지도 않았는데
아이폰8로 바꾸면서, 이어폰 구멍이 없어져서, 알아보게 됐네요~
가격이 좀 높은편이라 몇일 고민하다가, 그래도 기존 이어폰은
불편할거같아서 구매했는데, 생각보다 귀에서 떨어질 염려도 없는거같고
통화나, 음악 음질도 좋아요!
단점은, 블루투스로 해야된다는점, ^^ 4 

36 이어폰도 분실했고 차량용 블루투스도 불편해서 겸사겸사 비싼 이걸 주문했어요 사용해보니 확실히 편하긴하더라구요 귀에도 밀착감있게 착 붙어서 쉽게 떨어질듯하진 않고. 무엇보다 이어팟을 통해 듣는 소리가 되게 선명

75 십일절행사때 11일 오후1시쯤? 2주걸린다고되어있을때 주문했는데 17일 금요일 받았습니다~ 와 역시!!! 고민은 배송기간만 늦출뿐이라더니..하 진짜 이 말이 정답이네요... 너무좋아요ㅠㅠㅠㅠ아 진작살걸....정말후회됩니다....!!! 너~무 너무너무너무좋아요!! 착용감도 기존 이어팟보다 조금 더 가벼운느낌? 아무튼 편하고 음질도 아주 쥐똥만큼? 더좋은것같아요ㅋㅋㅋ느껴지네요 정~말 극 미세한 차이랄까 근데 확실히 좋아요ㅋㅋㅋㅋ더 낫습니다 평소에 저처럼 이어팟이 최고였고 이어팟만 쓰던분들은 엄청나게 만족하실거에요! 그리고 제조년월도 최신이에요ㅋㅋㅋ 2017년 11월이네요 좋아요♡ 감사합니다ㅋㅋ 5 

76 오 좋네요. 블루투스 이어폰은 처음인데 선 없는게 정말 편하고 좋네요. 음질도 생각보다 좋아요. 다 좋은데 한가지 단점은 가격이네요. 가격때문에 망설이다가 구입했는데 그래도 잘 한 것 같아요. 배송은 하루만에 바로 왔어요. 오픈형이라 귓속에 집어넣는거 싫으신 분들에게 적합합니다. 4 

77 해외배송으로 오는 다른 곳에서 주문했다가 물건 주문이 무기한 연기가 되서 11번가로 찾았는데 배송도 엄청 빠르고 제품도 2017년 제조 물건이고 안전하게 잘 왔습니다! 돈값을 하는거 같이 편리하기도 하고 음질도 좋네요!! 굿굿굿 5 

78 이어폰 줄때문에 스트레스 받아서 구입한건데 제 폰이 문제인건지...지하철에서 음악듣는데 노이즈가 너무 심합니다 전화할때도 유선이어폰과 달리 마이크가 에어팟 자체에 붙어있어서 듣는사람이 주변소음이 너무 크게들린다고 하네요 3 

79 배송짱빠름 블투이어폰 사려는데 이것도 뭔가 맘에안들고 저것도 뭔가 맘에안들고,, 이러다가 사는김에 그냥 크게 지르자 해서 에어팟구매 ! 11번가 무이자할부덕에 사는김에 애플워치도 같이 구매했음 원래 애플정품 이어폰보다 저음이 좀 빵빵해진거같고 블루투스이어폰 처음쓰는데 에어팟 진짜 짱인듯.. 5 

80 아이폰 사용자여서 이어폰잭이 없어 충전과 이어팟을 같이 못 써서 매우 불편해 큰 맘 먹고 샀네요. 이어

214 비싸긴 하지만..좋아요..소리가 약한듯.. 5 

215 역시작품이라 음질도좋고 사용편하네요 5 

216 음질도 품질도 좋습니다 5 

217 쓰는데 지장없고 음질좋고 좋네요 5 

218 음질 선명하고 커넥팅이 잘되요 4 

219 배송이 하루만에와서 굳선없이 노래 들으니까 어색ㅎㅏ지만 짱편해요!한쪽씩 터치 설정도되고,음질도 나쁘지않구요빠른배송 감사합니당~ 4 

220 배송 완전 빠르네요. 조카 졸업선물로 해줬는데 좋아하네요. 정품 맞구요. 음질도 좋다해요. 이래서 에어팟 하나봐요 5 

페이지 12 

221 너무 편하고 음질도 좋고 진작 구매할껄 그랬어요.
6만원짜리 이어폰 3일만에 분실ㅠㅠ 4 

222 롯데택배로 바로 보내니 배송도 빠르고 배송조회도 잘되고요 ㅎㅎ 신한카드에 엘포인트 등등 혜택도 많네요! 에어팟으로 삶의 질 올라가네요. 선없는 세상 최고에요! 쿵쿵 음질도 좋고요!! 아이폰이랑 페어링 잘되고 최고에요!! 5 

223 너무 간편하고 음질도 좋아요 최고입니다
고민끝에 비싼값을 지불하기는 했지만 막상 받고나니 만족도는 두배입니다 아깝지않아요 5 

224 배송 빨랐어요.에어팟 진짜 편하고 소리도 너무 좋아요. 5 

225 무선이라 무시했었는데 음질도좋고 역시에플 5 

226 적당한 배송과 적당한 가격에 구매한거 같습니다. 요즘 카피제품 많이 판매되는거 같던데 역시 정품이 좋은거 같네요. 음질 좋습니다. 3 

227 배송도 빠르고 음질도 좋아요!! 4 

228 음질좋고 괜찮습니다. 5 

229 생각보다 음질이 좋습니다. 이어팟 비교 중저음이 좋습니다. 구매하실분 참고 하세요~~ 4 

230 안드로이드 폰을 사용하지만 너무 쓰고싶어서 남편꺼랑 2개 주문했어요~ 잘 되고 정품등록도 됫어요 좋네요!음질도 좋고 싸게 살수 있어서 좋아요 5 

231 귀가 커서 조금 안맞긴 하지만 기능상으로나 너무 마음에 들어요 음질도 좋고 그리고 되게 편리해요 5 

232 음질도 좋고 통화음질도 좋네요.굿입니다.너무 맘에들어요. 적극추천합니다

345 배송도 하루만에 오고 에어팟 짱조아요 ㅎㅎㅎ
음질도 좋고 만족도 최고입니당 5 

346 음질도 생각보다 좋구 디자인도 귀엽고 좋은데자석부분에 검은색 철이 계속붙네여ㅜ 5 

347 배송도 빠르고 음질도 좋고 아주 잘들려요 정말 만족합니다! 5 

348 배송도 음질도 최고! 선물해야하는거라 온라인에서 주문해도 되나 걱정하면서 주문했는데 배송도 빠르고 음질도 상당히 좋네요! 4 

349 에어팟 음질도 좋고 배송도 빠릅니다 사업 번창하세요! 5 

350 음질도 생각보다 좋고 무엇보다 편리합니다. 추천 제조일자도 18년3월이네요 굿굿 5 

351 남자친구선물로 샀어요 ㅎㅎ
음질도 좋고 무엇보다 배송이 넘 빠르네요
감사합니다 4 

352 역시 다르다는 느낌.
소리가 우선 맘에 들고,
무선의 편리함과 siri의 편리함을 만끽합니다.
가격의 값어치 한다는... 5 

353 첨에 그냥 저렴한 일반 블투이어폰을 썼다가 너무 쉽게 고장나서 큰 맘 먹고 정품샀는데 너무너무 좋아요!!! 부드럽고 음질도 괜찮아요.. 크기도 부담스럽지 않고 배송도 엄청 빨라요. 추천합니다 5 

354 왜 에어팟에어팟 하는지 알겠어요 솔직히 너무좋아요

귀에서 잘 빠지지도 않고 가볍고 음질도 좋고 통화도 잘되고 !!추천합니다 구매하셔도 절대 후회안하실거에요 4 

355 깔끔하게 잘 보내주셔서 감사합니다 원래 이어팟 쓰던사람이라 역시 에어팟도 좋네요 음질도 괜찮고 성능도 문제없어요~ 타 사이트보다 싸게 구매할수 있어서도 좋았습니다 5 

356 배송도 완전 빠르고 음질도 완전 좋아요ㅋㅋㅋ완전 대만족ㅋㅋㅋ 완전 좋아욬ㅋㅋㅋ 재구매 의사율도 아주 높습니다 5 

357 빠른 배송~ 선이 없어서 무척 편합니다. 가격이 비싸다는게 단점이라면 단점이죠. 핸드폰 하고 거리가3m이내 벽이 하나 있으면 소리가 끊겨요~ㅎ (블루투스의 숙명??ㅋ) 여튼 편하고 좋네요. 4 

358 가격때문에 고민 많이 했는데
타 싸이트보다 저렴하고 무엇보다 당일 주문해서 당일날 받을 수 있어서 너무 좋아요~
생각

478 첨에 받자마자 설명서를 봤는데 전부다 영어 ㅋㅋ
친구덕분에 사용법 알아서 짐 넘 잘 쓰고있어요
끊김도 없고 음질 최고입니다
가격대비 훌륭해요 애플은 늘 사람을 중독시키는 매력이 무궁무진한거같아요 5 

479 왜 애플인지 다시 한번 느꼈습니다 왜 애플인지 다시 한번 느꼈습니다말이 필요 없습니다심플하면서도 완성도가 높고 음질도 괜찮습니다전 정말로 적극 추천!! 5 

480 에어팟. 이거. 정말. 편해요..!!귀에도. 딱맞고..운동할때. 딱이네요...!선. 걸리적거리는데..무선이라. 넘편하고 좋네요.음질도.생각했던거보다..괜찬아요...^^ 5 

페이지 25 

481 하루만에 왓습니다 음질도 좋구요 후회안합니다~ 5 

482 음질이 엄청 좋진않음12312145321351351351351354351 5 

483 음질은 솔직히 잘모르갯네요 이어팟보다 좋으시다고하는데 이어팟이 조금더 좋은거 같아요 그렇다고 음질이 나쁘다는건 아닙니다 4 

484 아주잘받았습니다. 음질도 너무좋고 이어폰기능도너무좋아요앞으로 또이용하고싶네요 ㅈ어말정말로 와 이렇게 좋은이어폰은 진짜또처음이네여 ㅋㅋ 다음에또 이용할께요 번청하세요 ^^화이팅이이 5 

485 애플과 호환성이 좋아 사용하기 편리하네요. 다른 블루투스 이어폰보다 음질또한 우수하고 좋아요. 강력 추천합니다.
복합할인 및 포인트 적립 그리고 롯데상품권까지 받게되면 저렴하게 잘산것 같아요. 4 

486 선없는거 진짜 좋네요. 에어팟이라서 아이폰하고 연동도 빠르고 음질도 좋네요! 5 

487 택배 진짜 빠르네요 ㅋ 가격비해 내가 원하는 음질이 아닐까봐 걱정했는데 음질 아주 마음에 듭니다 ㅎ 생각보다 스마트 하구요 강추합니다 5 

488 쇼킹딜때 저렴하게 구입해 2주정도 기다려야된다고하여 마음비우고 있었는데 12월16일 수령했어요.음질.디자인 다좋네요.너무만족스럽네요 5 

489 삼성폰 쓰고 있어서 겁정 많이 했는데, 블루투스로 연동도 잘되고 예뻐요, 음질도 좋네여~ 5 

490 편하고 음질도 좋아요 역시 가격이 

In [23]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

        type category                                             review  star
1    에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2    에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3    에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4    에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5    에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..       ...      ...                                                ...   ...
579  에어팟 1세대       음질                                좋습니다 무선의편리함 음질도 좋아요     5
580  에어팟 1세대       음질                                             음질 좋네요     4
581  에어팟 1세대       음질           에어팟 넘 편리하고 좋네요. 음질은 딱 이어팟 수준. 이건 좀 아쉽네요.     4
582  에어팟 1세대       음질                                       음질도 좋구 만족합니다     5
583  에어팟 1세대       음질                        역시 좋아요 에어팟 정말 편하고 음질도 좋습니다.     5

[3614 rows x 4 columns]


## 품질 리뷰

In [24]:
#"품질" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name=name[0]  #에어팟 1세대
category=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name,category,reviews,stars,cnt)
save()

페이지 1 

1 일단 에어팟 자체가 좋기 때문에 품질은 이상없고 좋아요.
주문한지 2일정도만에 배송되었어요.
배송도 빠르고 해당 판매업체에서도 최근에 생산된 제품을 보내주셨어요. 19년 3월에 생산된 제품으로요.
가격도 시중가보다 싸게 샀고요. 하나 더 구매할까 고민중이에요.
감사합니다. 5 

2 배송, 품질 모두 다 너무 좋았네요~ 사은품도 꼼꼼히 챙겨서 보내주셨습니당!! 게다가 2018년 10월 제조 제품!!!!!!! 이게 제일 기분 좋았던 것 같아요ㅎㅎ 에어팟 성능은 두말하면 잔소리죠?! 선물용으로 산 건데 받은 사람 준 사람 모두 만족하는 선택이었어요. 제것도 하나 살까 하는데 그때도 다시 이용할 계획입니다 ㅎ 재고가 부디 있기를 ㅠㅠ 5 

3 주문하였던 제품을 잘 받았습니다. 예상보다 택배가 아주빨리 도착하여 매우 좋았습니다.가격대비하여 품질이 좋은것 같습니다.디자인이나 기능면에서도 괜찮은것 같아서 추천이 가능한 상품이라고 할수 있겠습니다. 잘 사용하도록 하겠습니다.수고하세요. 5 

4 선물하려고 알아보다가 타 사이트가 1만원 정도 더 쌌지만 입고지연이라길래 하이마트몰에서 구매했는데 후회없어요!
결제 후 몇시간만에 상품이 출고되었고 정말 빨리와서 놀랄 정도였어요ㅎㅎ
빨리 필요하신분은 하이마트 몰에서 구매하세요! 시간이 돈이니까요!
상품 품질도 좋습니다! 잘 작동하고 깨끗한 상태로 배송왔어요!
나중에 다른 전자제품 구매할때 하이마트 또 이용해야겠어요! 5 

5 금요일 오후에 주문하고 결제했는데 토요일에 도착했어요!!!
쿠팡보다 빠른듯해요
가격도 인터넷 최저가 보면 배송비가 2만원정도 되었는데 하이마트는 저렴하고 배송비도 없고 믿을 수 있어서 넘 좋았어요!
역시나 에어팟 품질 말할것도 없고 꼼꼼하게 뽁뽁이로 포장해주셔서 감사합니다ㅜㅜ 하이마트 애용할게요! 5 

6 해외 직구했다가 기다리는 거 싫어서 취소하고 하이마트에서 구매했는데 배송 지-인짜 빠르고 토요일에 눈오는데 저녁에 배송해 주셨어요ㅠㅠ 품질도 너무 좋도 정품등록도 한방에-!! 삶의 질

In [26]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

        type category                                             review  star
1    에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2    에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3    에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4    에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5    에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..       ...      ...                                                ...   ...
112  에어팟 1세대       품질  배송빨라요 아주빨라요 일단 품질도좋고 생각보다 훨신 만족스럽네요 에어팟 강추 모양새...     5
113  에어팟 1세대       품질                                 가격 품질 배송 모두 만족 합니다     5
114  에어팟 1세대       품질                              배송도 빠르고 정품이라 품질도 좋네요!     5
115  에어팟 1세대       품질                                     배송도빠르고 품질도 좋아요     5
116  에어팟 1세대       품질             배송도 빠르고 제품 품질이 너무 좋아요선물용으로 하나 더 구매합니다!     5

[3730 rows x 4 columns]


## 성능 리뷰

In [27]:
#"성능" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name=name[0]  #에어팟 1세대
category=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name,category,reviews,stars,cnt)
save()

페이지 1 

1 우선 결제방법 바꾼다고 주문이 꼬였는데도 융통성있게 잘 보내주셔서 너무 감사하고, 제품도 포장꼼꼼히 안전하게 잘도착했습니다. 그리고 다른 사이트보다 혜택이 많아서 싸게 구입할수있었습니다.

여자친구한테 선물로 사줬는데 너무 좋아하는 모습을 보니, 정말 잘샀다는 생각이 드는군요, 그리고 점심때 주문했는데 다음날 오전에 바로 받아서 깜짝놀랐습니다 너무 빠르네요.

물건상태나 bb 잘쓰고 다음에 또 주문하겠습니다. 하이마트 짱짱!!! 5 

2 즐겁게 사용하고 있어요 걸을때 시간 보내야 될때 일하면서 손이 자유롭지 않을때 여러 상황에서
잘 사용되더군요.수신감도 좋고요 많은 분들이 쓰는 이유가 있는 제품 배송도 주문 결제 후 1~2만에 안전하게 제 손에 들어왔네요ㅎㅎㅎ 5 

3 갑자기 에어팟에 꽂혀가지고 여기저기 알아보다가 하이마트에서 파는걸 발견했는데 청구할인이랑 l포인트도 적립되고해서 나쁘지않은 금액으로 잘 산 것 같아요!ㅎㅎㅎ
배송도 빨라서 좋았구요 꼼꼼하게 잘 포장해서 주셔서 감사해요~! 에어팟은 성능도 좋고 잘 작동하구요 그냥 보기만해도 너무 귀엽고 예쁘네요ㅎㅎㅎ계속 끼고있고 싶어요~~ 착용샷도 괜찮고! 진짜 비싸서 고민 많이했는데 그만큼 만족감도 커서ㅋㅋㅋㅋ주변사람들한테도 추천중이에요!!ㅋㅋㅋ 5 

4 사용 후 이틀 뒤에 후기 남겨드립니다.솔직히 말씀드리자면 너무 좋습니다. 이어폰을 빼서 귀에 꽂자마자 아이폰과 연동되서 쓸 수 있는게 너무 편하고, 전화가 오면 두번 터치해서 통화가 되는 것도 너무 좋습니다. 통화음질 또한 잘 들리고 잘 전달됩니다. 조금 가격이 비싼게 흠이지만 기계 성능으로만 봐서는 정말 만족스럽습니다. 나중에 제 여친도 꼭 사주고 싶네요. 5 

5 디자인이야 두말 할것 없이 깔끔 그 자체예요. 얼핏보면 귀걸이 한것 처럼 보이기도 ㅋㅋ 기기상에는 작동 버튼이 없어 통화하거나 볼륨을 조절하거나 할때는 핸드폰으로 해야 하는 불편은 있지만 반응이 엄청 빠르고 통화 음질 아주 좋고 디자인이 예뻐 불편함은 패스 ㅋㅋ 무선이고 

94 배송 진짜 빨리 왔네요. 이틀만에 받았어요. 일마존 복잡해서 11번가에서 구매했는데정품으로 잘 받았고. 충전량이 95%정도는 돼 있었네요. 바로 사용가능했습니다. 에어팟 자체의 성능도 좋아요 5 

95 추천해여 완전 쩔어여 콩나물ㄴ낌없어여무조건 애플쓰는 사람은 구입해야할 에어팟 페어링도 빠르고 안드로이드 컴퓨터 할거없이 잘됩니다 5 

96 성능좋아요~신랑 선물로 사 주었네요~만족 5 

97 디자인은 별로데 성능은 생각보다 괜찮음 3 

98 예상대로 역시나 디자인 좋고 성능 좋고 음질도 좋네여
맘에 드는 제품입이다 5 

99 기능이나 성능은 맘에 듭니다 4 

100 싸게잘샀습니다 성능도좋습니다 5 

페이지 6 

101 착용감도 좋고~ 선이 없어서 편하고 좋음
단 블루투스라 지하철에서 끊기는 현상이 발생함 5 

102 빠른 배송과 더불어 좋은 성능 까지 대단히 만족 스럽습니다. 5 

103 생각하신 그대로 에어팟
성능좋고 가격비싸고 3 

104 상품자체는 나무랄점이없습니다. 블루투스, 이어폰 더블 터치 인식, 통화음질, 부착성 다 만족입니다. 다만 실리콘케이스는 꼭사세요. 그립감이 틀려요 ㅎ 5 

105 성능은 말할것도 없이 좋고 배송도 생각보다 빨리왔네요 5 

106 신랑이 사달라고 해서 최저가를 확인하던중 이곳에서 구입하게 되었네요.제일 저렴하게 구입한거 같구요.케이스부터 본체까지 너무 마음에 들고요.성능도 최상입니다.편하게 잘 사용할께요 5 

107 감사합니다 배송진짜 빨라염 성능도 매우 좋구여 역시! 5 

108 생일 선물로 구매했는데 너무 기분이
좋아요 찾기 쉽지 않았었는데 이렇게 구매를 하게 되어서요 그리고 사용감이나 주위의 반응도 좋습니다 5 

109 연결도 빠르고 배송도 빠르고 편의성 최강에 음질도 좋아요 진짜 사는거 후회 절대 안합니다! 5 

110 배송도 빠르고 이어폰 성능도 너무 좋았어요 ㅎㅎ
유선이어폰만 쓰다가 무선 블루투스 이어폰 쓰니까 너무 편하고 좋네요^^ 5 

111 좋은 상품 저렴하게 구매할 수 

In [29]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

        type category                                             review  star
1    에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2    에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3    에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4    에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5    에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..       ...      ...                                                ...   ...
107  에어팟 1세대       성능                      감사합니다 배송진짜 빨라염 성능도 매우 좋구여 역시!     5
108  에어팟 1세대       성능  생일 선물로 구매했는데 너무 기분이\n좋아요 찾기 쉽지 않았었는데 이렇게 구매를 하...     5
109  에어팟 1세대       성능  연결도 빠르고 배송도 빠르고 편의성 최강에 음질도 좋아요 진짜 사는거 후회 절대 안...     5
110  에어팟 1세대       성능  배송도 빠르고 이어폰 성능도 너무 좋았어요 ㅎㅎ\n유선이어폰만 쓰다가 무선 블루투스...     5
111  에어팟 1세대       성능  좋은 상품 저렴하게 구매할 수 있어 너무 좋습니다. 배송도 빠르고 주위 반응도 모두...     5

[3841 rows x 4 columns]


## 기능 리뷰

In [30]:
#"기능" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name=name[0]  #에어팟 1세대
category=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name,category,reviews,stars,cnt)
save()

페이지 1 

1 같은 행사를 롯데 포인트에서도 하길래 고민했는데 하이마트가 좀 더 싸고 무이자 할부도 돼서 드디어 질렀어요~~~!
커널형 이어폰만 써와서 계속 고민하다가 싸게 나왔길래 엘 포인트 갖고 있던 거 탈탈 털어서 총 10만원 정도에 샀네요
엘포인트 안 썼어도 16만원 대로 사는 건데 일마존 보다 싸요~!
배송도 진짜 빠르게 왔고 받자 마자 뜯어서 오픈하니 페어링 바로 되고...
감동적인 콩나물입니다.
커널형이 아니라서 주위가 시끄러우면 볼륨을 많이 높여야하는게 단점이긴하지만 선이 없어서 넘나 편해요!!!!
귀 안 건드리면 절대 빠지지도 않습니다.
고민하시는 분들 그냥 빨리 지르세요
고민은 배송만 늦어지게 할 뿐.... 5 

2 몇일전 다른 쇼핑몰에서 17만원대까지 내려갔으나 주문폭주로 배송이 4주까지 걸린다는 내용을 보고 몇만원 더 주고 빨리 받고 싶어서 찾아보니 19만원대에 판매하는 당일배송인 타메몰 제품을 찾았습니다 후기가 없어서 고민하던 중 우선 결제부터 했더니 바로 당일배송해주셔서 다음날에 잘 받아볼 수 있었어요 상품 기능 정상적으로 다 잘되구요 더블탭 기능이 처음엔 잘안됐으나 쓰다보니 인식이 잘돼서 너무 유용하게 쓰고 있네요 다음곡 기능 최고에요 애플의 소프트웨어 업데이트로 볼륨 업다운도 넣어주길 바래봅니다 다시 한번 빠른 배송과 좋은 상품 그리고 안전한 배송(든든한 에어캡포장)에 감사드립니다 다른분들 리뷰처럼 생활의 질이 올라간 기분 최고네요 추가적으로, 안드로이드 폰과의 궁합도 나쁘지 않습니다 5 

3 주문하였던 제품을 잘 받았습니다. 예상보다 택배가 아주빨리 도착하여 매우 좋았습니다.가격대비하여 품질이 좋은것 같습니다.디자인이나 기능면에서도 괜찮은것 같아서 추천이 가능한 상품이라고 할수 있겠습니다. 잘 사용하도록 하겠습니다.수고하세요. 5 

4 많은 리뷰에서 높게 평가했듯이 동일 iCloud 계정을 공유한 디바이스들 사이의 연결성이 꽤 좋습니다. 디바이스와 의 첫 연결은 매우 심플하고 어찌보면 꽤 세련된 방식을 사용하고 있습니다.

In [32]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

       type category                                             review  star
1   에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2   에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3   에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4   에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5   에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..      ...      ...                                                ...   ...
66  에어팟 1세대       기능         쓰면 알겁니다 엄청 좋네요 바로 페어링이 돼서 편하고 음질도 괜찮아요 추천!     5
67  에어팟 1세대       기능  맥북 아이패드 사용중이라 구매 했는데 페어링도 편하고 착용감도 편해요. 폰은 s7쓰...     5
68  에어팟 1세대       기능  배송은 진짜 빠릅니다. 에어팟 기능도 만족하고요. 다만, 옵션으로 구매한 케이스 퀄...     5
69  에어팟 1세대       기능        착용감,음질,기능 모두 만족합니다. 여러분들이 좋다고 하는데는 이유가 있네요.     5
70  에어팟 1세대       기능               좋다네요 저는 아직 다른걸 사용해서\n그런데 편위 기능이 좋습니다     4

[3911 rows x 4 columns]


## 배터리 수명 리뷰

In [33]:
#"배터리 수명" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name=name[0]  #에어팟 1세대
category=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name,category,reviews,stars,cnt)
save()

페이지 1 

1 에어팟 좋네요. 가격이 다른 블루투스 이어폰에 비해서 비싸다는 생각을 했는데, 타사 제품에 비해 확실히 배터리가 오래가구요. 사용성이 정말 좋은것같아요. 귀에 꽂으면 인식돼서 소리 나오고 빼면 꺼지고, 두드리는 모션 인식 되고 이런 작은 것들이 편리해요! 5 

2 사실 다른 코드프리 이어폰들과 비교해서 비싼 가격이라고는 생각되지 않습니다.
다른 제품들을 많이 사용했지만 에어팟만큼 완성도가 높고 편리한 제품은 많지 않다고 생각합니다.
다른 애플기기들과 사용할 때에는 그 어떤 이어폰들이 상대가 되지 않을 정도로 편리합니다.
음질도 블루투스로서는 만족할만한 수준이고, 배터리도 오래가고, 충전할수 있는 케이스에 탈착하는 것도 다른 이어폰들보다 정말 편하고 깔끔합니다. 5 

3 올충전하고 블루투스 연결하고 귀에 꽂았을?? 바로 연결은 안되요 1-2분정도 딜레이가 있습니다.그리고 생각보다 이어폰 배터리가 오래가지않는것같음. 안드로이드는 쓰지마시고 아이폰만쓰세요안드로이드는 호환안되는거 많아서 ㅂㄹ임 3 

4 아이폰 가진 사람은 무조건 사야합니다. 구매하고 사용해보니 이건 혁신입니다. 선하나 없는 깔끔한 착용에다가 무게도 가벼워서 찬듯 안찬듯한 느낌이 납니다. 단점이라 한다면 에어팟 착용하고 통화를 하면 사람들이 이상하게 쳐다봅니다. 혼잣말하는 정신병자인가?라는 눈초리를 받기 때문에 이게 단점이고 에어팟 착용하고 시리 소환 후 전화도 가능하고 정말 편리하네요. 생각보다 밧데리도 오래가구요. 그리고 밧데리 소모되도라도 충전케이스에 넣어두면 금방 충전되기때문에 지하철이나 버스 탈때 사용하시는 거라면 밧데리 소모 걱정은 안하셔도 됩니다. 큰 기대없이 좋은 기회라 생각되어 구입했는데 대 만족입니다. 5 

5 망설였는데, 써보니까, 생각보단 훨씬 좋습니다. 사람이 많은 곳에선 디바이스가 에어팟에 가까이 있어야 끊김이 덜하는 건 있네요. 편하고 배터리도 생각보다 오래가서 좋습니다. 볼륨과 선곡이 디바이스에서만 되는 부분은 조금 불편한 면이 있긴 하네요, 그래도

In [35]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

       type category                                             review  star
1   에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2   에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3   에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4   에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5   에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..      ...      ...                                                ...   ...
6   에어팟 1세대   배터리 수명  진짜....3만원때 저가 제품도 써보고 8만원때 제품도 써봤지만....지하철에서 끈...     5
7   에어팟 1세대   배터리 수명  연결빠릿하고 배터리 오래가고 통화품질도 좋아요되게 하얗고 맨들맨들해서 좋은데 기스 ...     5
8   에어팟 1세대   배터리 수명                         너무 좋습니다 배터리도오래가고 음악도 잘들리네요     5
9   에어팟 1세대   배터리 수명                    아 완전편합니다 생각보다 배터리도오래가고 아이폰에 강추!     5
10  에어팟 1세대   배터리 수명    한달정도 사용중인데 너무 편하고 좋아요 ㅎㅎㅎ 음질도 좋고 배터리도 오래가고 좋습니다     5

[3921 rows x 4 columns]


## 에어팟 2세대

In [43]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

#에어팟 1세대 검색 (2세대 검색하고싶으면 주소바꾸셈)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=18754856889&query=%25EC%2595%25A0%25ED%2594%258C%2520%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25202%25EC%2584%25B8%25EB%258C%2580&NaPm=ct%3Dkb505jgo%7Cci%3D86a51f76becbcec6a02c0670a35d66067166cf23%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D1c9c317e06ee8905a7b4c2f4b21074e01221873a"
#xpath
shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[21]/a/span" #배터리 소모

# name="에어팟 1세대" #에어팟 2세대 검색할 때 2세대로 바꾸셈
# category1_="음질"
# category2_="품질"
# category3_="성능"
# category4_="기능"
# category5_="배터리 수명"
# category6_="배터리 소모" 

header = {'User-Agent': ''}
d = webdriver.Chrome(executable_path=r"C:\Users\jmlee\Code\chromedriver_win32\chromedriver.exe") # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

In [44]:
def add_dataframe2(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df2=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df2.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df2.loc[n]=[name,category,'null','null']
        n+=1    
    return df2

In [50]:
# .to_csv 
# 최초 생성 이후 mode는 append
import pandas as pd
import os
def save():
    if not os.path.exists('output.csv'):
        df2.to_csv('output.csv', mode='w',encoding='utf-8')
    else:
        df2.to_csv('output.csv',mode='a',encoding='utf-8',header=False)

In [45]:
#"음질" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name=name[1]  #에어팟 2세대
category=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe2(name,category,reviews,stars,cnt)
save()

페이지 1 

1 배송도 빠르고 정품인것 같네요^^
근데 정품등록이 안되서;; 안해도 별상관은 없다지만 그게 좀 아쉽네요^^
제품자체는 맘에듭니다. 페어링도 잘되구요. 근데 마감이 좀 안좋아서 귀안쪽으로 깊이 넣으면 긁혀서 아픕니다ㅠㅠ.
그래도 두손이 자유로워져서 너무 편하고 통화품질도 좋은것 같아서 만족합니다! 5 

2 음질이 생가보다 좋았어요. 특히 베이스가 빵빵한것이 맘에 듭니다. 무선이라 편하고요. 갤럭시라 따른 앱을 받아서 설정을 했는데 배터리가 좀 빨리 다는것같았어요. ff기능은 터치말고 버튼이나 모 이런걸로 했으면 더 좋았을거같은데 그래도 쓸만하더라구요. 5 

3 카드사 행사때 저렴하게 구매해서 가격이 만족스럽습니다 여러 키링과 에어팟케이스로 꾸며서 예쁜 백참처럼 이용하는 재미도 좋습니다. 무선이어폰은 정말 신세계네요 각인서비스를 이용하지 못하는 단점이 있기는 한데 뭐 보통 에어팟케이스를 쓰니 구지 각인서비스가 필요는 없겠다 싶어서 그냥 구매했어요 단지 현재 이용하는 빈도에 비해 배터리가 빨리 닳아서 애플센터에 한번 방문해보려구요 뽑기가 잘못된건지 좀 확인해 봐야겠네요 가격은 좀 비싸지만 가급적 아이폰은 제 짝꿍인 에어팟을 이용하는게 개인적인 느낌에서 좋은것 같습니다 음질도 너무 좋아요 그리고 어느정도 거리까지는 멀리서도 이어폰 이용이 되니 너무 편리합니다. 카드사 행사나 쿠폰등 이용하면 좋은 가격에 살수 있으니 추천합니다. 유선충전과 무선충전 다소 고민했는데 무선충전이 확실히 편리함이 있습니다. 몇마넌 차이가 나지 않기때문에 . 약간 귀에서 떨어져서 잃어버릴수도 있겠다 싶긴 한데 요즘 예쁜 색상 이어캡이 많아서 꾸미기도 할겸 실용성 있게 구매해 이용해 보려고 합니다. 5 

4 유무선 중 고민하다 무선으로 구매했어요. 뭐 음질이 너무 좋다해서 기대 많이 했는데, 사용하고 있는 유선 이어폰하고 비교해도 월등하다고 할 정도는 아니네요. 가격따지면, 기대만큼 지불할 정도 아닙니다. 무선쓰다 유선 쓰니 불편해서 사긴 했는데요. 사실 분들, 에어팟 구

112 노이즈가 심해서 애플가사 교환했네요.... 5 

113 음질이 너무 좋네요. 디자인과 컬러 스타일이 아이폰스럽네요 5 

114 빠른배송 감사합니다. 음질도 좋네요 5 

115 음질이 정말 좋아요~ 4 

116 고심한 끝에 결국 질렀습니다
역시 고민은 구매만 늦출뿐이네요
귀에서 이어폰 잘 이탈하는 편이라 고민 했었는데
막상 받아서 착용해보니 귀에 착 감기고 음질도 좋습니다
배송도 빠르고 안전히 왔고 마음에 듭니다 5 

117 빠른배송에 한번 감동, 제품 성능에 또 감동 했어요생각보다 귀에 답답한 느낌없이 잘들리고 좋아요 음질도 굿 5 

118 음질도 좋고 편하고 완전 만족합니다.배송도 무지 빨랐구요^^ 5 

119 자주 애용할게요
음질도 짱좋아요 잘샀네요 5 

120 그동안 고민했던 게 시간이 아까울 정도네요
무선이어폰은 정말 신세계네요
음질도 좋고
가격은 하이마트가 젤 쌌어요!!
배송도 빨라서 아주 만족합니다 5 

페이지 7 

121 배송도 빠르고 음질도 좋아요 5 

122 음질좋고 짱 입니다 .. 5 

123 갑자기 한쪽이 안들려서 잠시 당황했지만 인터넷서치해서 고쳤네요. 이런 문제만 안 생기면 음질도 좋고 다 괜찮아요. 남편꺼 하나더 사러 들어왔네요 4 

124 무선충전도 잘되고
음질 좋아요
통화도 제일 나은거 같음 5 

125 중2 아들이 용돈모아서 산 에어팟입니다!
비싸서 다른제품과 고민 많이 해서 산것같은데ᆢ
역쉬~~ 굿초이스🤗
오자마자 엄청 좋아하네요!
음질도ᆢ 굿! 간지납니다~ 5 

126 근데터치잘안되고구린것가타여
진심불량인가싶더라구요
음질은좋고사용감이줄없어서편함 3 

127 역시 애플입니다! 음질 및 편의성이 넘 좋아요~ ㅅ ㅏㅇ품 잘받았습니다~ 5 

128 별 설정 없이 아이폰에 바로 인식되네요.
음질도 훌륭합니다.
모처럼 아빠노릇 한번 제대로 했네요. ^^ 5 

129 배송 정말 빠르네요.음질 정말 좋습니다. 5 

130 배송도 빠르고 음질도 너무 좋아요~ 5 

131 선
넘 맘에 들어요...ㅋㅋㅋ

In [52]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

        type category                                             review  star
1    에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2    에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3    에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4    에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5    에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..       ...      ...                                                ...   ...
131  에어팟 2세대       음질  선\n넘 맘에 들어요...ㅋㅋㅋㅋㅋ\n선 없는 게 이리 편할 줄이야ㅜㅜ\n무선충전도...     5
132  에어팟 2세대       음질                                 빨리 와서 좋고 음질도 좋고 !!     5
133  에어팟 2세대       음질  살까말까 고민하신다면 사세요 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ처음엔 에어팟보고 저 콩나물 ...     5
134  에어팟 2세대       음질  음 제가 샀을때보다 가격이 더 떨어져서 마음 아프네요ㅠㅠ\n그래도 상태도 음질도 좋...     5
135  에어팟 2세대       음질  배송도 우첵국 택배로 하루만에 배달됬고, 지금까지 2주됬는데\n아주 잘쓰고 있습니다...     4

[4062 rows x 4 columns]


In [53]:
#"품질" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name=name[1]  #에어팟 2세대
category=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe2(name,category,reviews,stars,cnt)
save()

페이지 1 

1 에어팟2 구매하고 며칠 사용해보니 진작에 살 걸 그랬다는 생각이 많이 듭니다.
다른 걸 다 떠나서 선이 없는 자유로움이 주는 편리함이 정말 최고네요.
물론 외형도 작고 심플하고 예쁘기도 하고요.
케이스는 없이 사용하다가 오늘 민트컬러 실리콘케이스 끼워봤어요 :)
설명대로 충전케이스 한번 충전하면 유닛 사용하고 배터리 떨어져도 다시 끼우면 바로 충전되니까 꽤 오래 사용할 수 있어서 편해요.
물론 아이폰 충전하랴 에어팟케이스 충전하랴 더 할 일이 늘어나서 번거로운 느낌은 있는데 에어팟은 매일 충전할 필요는 없으니까요.
음질은 기존 이어폰이랑 별다를 거 없이 평범한 느낌이지만 에어팟은 편의성에 더 중점을 둔 제품이니까요.
이중 탭으로 유닛제어가 가능하니 아이폰 꺼낼 필요도 없고 전화 받는 거도 통화도 잘돼요 두 손이 정말 자유로워졌어요.
사용하다보니 노캔 기능이 있으면 좋았겠다 싶은데 그럴 거면 프로를 구매했어야 하지 싶네요.
구매한 지 얼마 안 되었으니 열심히 사용해줘야지요. 아껴가면서.
노캔빼고는 완벽하니까 망설이는 분들 강추합니다 :) 5 

2 배송 진짜 빨라요. 문의에 대한 답도 빠르고 정품 등록 고객센터에 할 때 필요한 서류도 잘 보내주시네요. 어디서 살지 많이 망설였는데 잘 산 것 같아요. 제것도 구입한다면 재구매 할 것 같네요! 디자인 정말 앙증맞고 귀여워요. 성능이야 말할 것도 없그요. 5 

3 본인 막귀임. 이어폰 여태 사 본 중에 제일 비싼게 2만5천원임.
에어팟 처음 나왔을땐 디자인 맘에 안들어서 무시했었음. 그러다가 통화품질이 넘사벽이란 얘기를 들음. 솔깃함.
근데 곧 2세대가 나온다는 소리가 있어서 참음.
2세대 나오고 나서 보니 1세대 대비 큰 변화를 느끼지 못해서 고민함. 그러나 통화품질 관련해서 전문가 리뷰보고 사야겠다 결심함. 노래 품질은 그저그럼. 근데 통화할땐 전화기 들고 하는거 마냥 상대방 목소리가 선명하게 들림. 상대방도 이어폰으로 통화하는지 모를만큼 목소리가 선명하게 들린다함. 근데 밖에서 통화하면서 차지

In [54]:
#"성능" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name=name[1]  #에어팟 2세대
category=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe2(name,category,reviews,stars,cnt)
save()

페이지 1 

1 제품 개봉했을 때 사진과 같은 결함이 있었고 판매자와 통화 후 고객센터 통화 서비스센터를 방문했고 답변은 "제품 초기 외관 불량은 판매처에서 책임을 지는게 맞다." 이었고 판매자에게 다시 전화해서 말씀드렸더니 처음부터 끝까지 죄송하다는 말은 한번도 안하고 본인들은 판매글에 공지를 했고 구매자도 거기에 동의를 했으니 구입하신게 아니냐 고 말하면서 결국 본인들은 책임이 없다네요~~ 판매자의 말투에 정말 불쾌했고 결국 하신 사과는 제 말 툭 끊고 "아 고객님 제가 정말 정중하게 사과드릴게요! 정말 죄송합니다!!" 하시더라구요~ 사람 비꼬세요?ㅠㅠ 저는 저 결함 때문에 전화드린건데 자꾸 저를 말귀 못알아먹는 사람 취급하면서 예를들면~ 고객님이 자동차를 구매했을때~ 제품이 파손됐을때~ 하시면서 저를 이해시키려고 하더군요 리뷰가 1000자 까지 밖에 안써져서 정말 아쉽습니다. 글짖기 마냥 제가 겪은 일 주절주절할 수 있는데.
같은 말을 해도 다르게 전달 될 수 있는거에요~ 고객센터에선 전화받자마자 아 죄송합니다 고객님 많이 속상하셨겠어요 하면서 응대해요~ 이게 구매자한테 서비스를 제공하는 판매자가 갖춰야 할 태도라고 생각합니다. 통화중에 자꾸 어이없다는 듯이 코웃음 치고 비꼬고 말끊고 하지 마시구요 ㅎㅎ 아! 긴얘기 끝에 고객님이 제품을 보내주시면 저희가 리퍼 요청을 넣어드릴순 있다 라고 말하긴 했어요! 아주 긴~얘기 끝에 겨우요!
이 제품을 구매할 소비자가 있으시다면 신중히 고민하시길 바래요~ 판매자한테 조롱받은 저는 너무 속상하네요~ㅠㅠ 1 

2 유무선 중 고민하다 무선으로 구매했어요. 뭐 음질이 너무 좋다해서 기대 많이 했는데, 사용하고 있는 유선 이어폰하고 비교해도 월등하다고 할 정도는 아니네요. 가격따지면, 기대만큼 지불할 정도 아닙니다. 무선쓰다 유선 쓰니 불편해서 사긴 했는데요. 사실 분들, 에어팟 구매 신중히들 하세요. 여러 제품 써봤지만, 가격대비 훌륭한 제품 많습니다. 20만 원 가까이 주고 살 물건은 아닌 거 같아요. 품질점수:그냥

In [55]:
#"기능" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name=name[1]  #에어팟 2세대
category=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe2(name,category,reviews,stars,cnt)
save()

페이지 1 

1 본인 막귀임. 이어폰 여태 사 본 중에 제일 비싼게 2만5천원임.
에어팟 처음 나왔을땐 디자인 맘에 안들어서 무시했었음. 그러다가 통화품질이 넘사벽이란 얘기를 들음. 솔깃함.
근데 곧 2세대가 나온다는 소리가 있어서 참음.
2세대 나오고 나서 보니 1세대 대비 큰 변화를 느끼지 못해서 고민함. 그러나 통화품질 관련해서 전문가 리뷰보고 사야겠다 결심함. 노래 품질은 그저그럼. 근데 통화할땐 전화기 들고 하는거 마냥 상대방 목소리가 선명하게 들림. 상대방도 이어폰으로 통화하는지 모를만큼 목소리가 선명하게 들린다함. 근데 밖에서 통화하면서 차지나가는 소리. 오토바이소리 등 별로 크지도 않은 소리였는데 통화하는 상대방한테는 찢어지는 듯한 소음으로 들린다고함. 오히려 단점이된거같음. 무선충전 생각보다 발열 좀 있음. 겨울에 손난로로 쓰면 될거같음. 페어링잘되는데 시리야 버그는 나도 피해갈 수 없었음. 폰 네트워크 재설정으로 해결. 근데 페어링 후에 아이폰 이름이 누구누구의 아이폰에서 그냥 아이폰으로 지맘대로 바뀜. 자잘한 버그들이 아직 좀 있는듯. 에어팟 자체는 괜찮음. 깔끔하고 작고 가볍고. 청음해볼 수 있었으면 안샀을거 같긴하지만.
애플기기들과의 연동은 칼같아서 좋음. 워치 연결해서 밖에 있다가 집에 돌아오니 통화중이었는데도 지맘대로 폰이랑 . 당황스러웠음.
결론은 사면 후회할 정도는 아님.
근데 빵빵한 사운드를 기대한다거나 세상마상 완벽한 이어폰을 원한다면 살 물건은 아님. ㅇㅇ 5 

2 무선을 저렴하게 구입할 수 있어서 너무 행복해요정품등록에서 구매일이 제대로 안되서 그날 한달 전에 산걸로 되버렸지만.. 그래도 성능도 괜찮고 딱히 고장난 부분도 고쳐야할 부분도 뭣도 없고 깨끗하게 잘 와서 너무 좋아요! 오늘까지 세일인거같은데 꼭 사세요!!! 5 

3 배송은 신속 했구요.ㅎ 패키지는 늘 아름답습니다. 가격의 압박에 저렴이 블루투스 이어폰을 구입 하고 단 하루만에 이 녀석으로 다시 주문 했어요. 아이폰 맥북 유저인데 페어링 정말 심플하고 음질도

In [56]:
#"배터리 수명" 관련 리뷰 가져오기
name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','배터리 소모']
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name=name[1]  #에어팟 2세대
category=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe2(name,category,reviews,stars,cnt)
save()

페이지 1 

1 동생꺼랑 같이샀는데 음량도 좋고 작동도 잘되고 좋아요ㅎㅎㅎㅎ버즈랑 고민하다가 샀는데 잘산거같아요~ 4 

2 아이 선물로 사주었는데 음량 짱짱하니 좋다 하네요
배송은 진짜 빠르네요~ 5 

3 에어팟 무선 음량도 괜찮고 맘에들어요 귀가 좀 아프지만 적응되면 괜찮을거같아요 ㅎㅎ 5 



In [59]:
dataset = pd.read_csv("output.csv", index_col=0)
print(dataset)

       type category                                             review  star
1   에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
2   에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
3   에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
4   에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
5   에어팟 1세대       음질  배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 ...     5
..      ...      ...                                                ...   ...
29  에어팟 2세대       기능                                    배송도 빠르고 성능도 좋아요     5
1   에어팟 2세대   배터리 수명  동생꺼랑 같이샀는데 음량도 좋고 작동도 잘되고 좋아요ㅎㅎㅎㅎ버즈랑 고민하다가 샀는데...     4
2   에어팟 2세대   배터리 수명          아이 선물로 사주었는데 음량 짱짱하니 좋다 하네요\n배송은 진짜 빠르네요~     5
3   에어팟 2세대   배터리 수명     에어팟 무선 음량도 괜찮고 맘에들어요 귀가 좀 아프지만 적응되면 괜찮을거같아요 ㅎㅎ     5
1   에어팟 2세대   배터리 소모  제가 노트8을 쓰고있어서 에어팟을 사는게 좋을지 버즈를 사는게 좋을지가 첫번째 고민...     5

[4188 rows x 4 columns]
